In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
df = pd.read_excel("pps.xlsx", usecols='C,R:X')
df

,연도,연구 키워드,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1525,2021.0,위험도 지표,노출변수,상관분석,인구자동차계수,자동차도로계수,NaN,NaN
1526,2021.0,수요분석,게임이론,지역간 철도,대중교통 만족도,대기시간,NaN,NaN
1527,2021.0,사업용 자동차,위계적 모형,내재적 사고요인,로지스틱 회귀분석,운수종사자,NaN,NaN
1528,2021.0,jerk,분류기,SDI,서포트벡터머신,교통안전대체지표,NaN,NaN


# Task
년도별 논문 키워드의 빈도와 비율을 추출한다.

## Solution
`pd.DataFrame`의 Split-Apply-Combine 방식을 따른다.
* Step 1. 데이터프레임을 기준년도 2개를 조건으로하여 자료를 재분배
* Step 2. `collections.Counter` 객체를 활용하여 년도별 키워드를 카운트 및 정렬
* Step 3. 상위 20개 키워드만 추출 후 `pd.DataFrame`으로 저장

In [3]:
ls = {}
COUNT = {}

for i in range(2000, 2022, 1):
    ls[i] = df[df['연도'] == i].iloc[:, 1:].values.ravel()
    COUNT[i] = df[df['연도'] == i].shape[0]

In [4]:
len(ls)

22

In [5]:
COUNT

{2000: 52,
 2001: 64,
 2002: 96,
 2003: 60,
 2004: 96,
 2005: 106,
 2006: 90,
 2007: 86,
 2008: 116,
 2009: 108,
 2010: 90,
 2011: 72,
 2012: 60,
 2013: 48,
 2014: 65,
 2015: 55,
 2016: 44,
 2017: 42,
 2018: 40,
 2019: 40,
 2020: 39,
 2021: 60}

## Count keywords

In [6]:
counter_dc = {}

for year, content in ls.items():
    words = content[pd.notnull(content)]
    words_counter = Counter(words)
    counter_dc[year] = words_counter.most_common(20)

In [7]:
counter_dc.get(2021)

[('주행 시뮬레이터', 3),
 ('자율주행차', 3),
 ('C-ITS', 2),
 ('교통안전', 2),
 ('VISSIM', 2),
 ('고속도로', 2),
 ('모빌리티 빅데이터', 2),
 ('수단선택모형', 2),
 ('긴급차량', 2),
 ('인공지능', 2),
 ('V2X 통신', 2),
 ('코로나19', 2),
 ('자율협력주행', 2),
 ('머신러닝', 2),
 ('대중교통', 2),
 ('통행시간', 2),
 ('순응률', 1),
 ('사고 발생 개연성', 1),
 ('주행행태', 1),
 ('차량 내 경고 정보 서비스', 1)]

In [8]:
pd.DataFrame(counter_dc).to_csv("counter.csv", encoding='euc-kr', index=False)

## Calculate proportion

In [9]:
prop_dc = counter_dc.copy()

In [10]:
prop_dc[2009][0][1] # 년도, idx, 개수

6

In [11]:
for i in range(2000, 2022, 1):
    for j in range(len(prop_dc[i])):
        prop_dc[i][j] = (prop_dc[i][j][0], prop_dc[i][j][1], COUNT[i], round(prop_dc[i][j][1]/COUNT[i], 3)) # tuple can't be assigned

In [12]:
prop_dc.get(2021)

[('주행 시뮬레이터', 3, 60, 0.05),
 ('자율주행차', 3, 60, 0.05),
 ('C-ITS', 2, 60, 0.033),
 ('교통안전', 2, 60, 0.033),
 ('VISSIM', 2, 60, 0.033),
 ('고속도로', 2, 60, 0.033),
 ('모빌리티 빅데이터', 2, 60, 0.033),
 ('수단선택모형', 2, 60, 0.033),
 ('긴급차량', 2, 60, 0.033),
 ('인공지능', 2, 60, 0.033),
 ('V2X 통신', 2, 60, 0.033),
 ('코로나19', 2, 60, 0.033),
 ('자율협력주행', 2, 60, 0.033),
 ('머신러닝', 2, 60, 0.033),
 ('대중교통', 2, 60, 0.033),
 ('통행시간', 2, 60, 0.033),
 ('순응률', 1, 60, 0.017),
 ('사고 발생 개연성', 1, 60, 0.017),
 ('주행행태', 1, 60, 0.017),
 ('차량 내 경고 정보 서비스', 1, 60, 0.017)]

In [13]:
pd.DataFrame(prop_dc).to_csv("prop.csv", encoding='euc-kr', index=False)

## Minor tasks

In [14]:
# only keyword
kw_dc = {}

for year, content in ls.items():
    words = content[pd.notnull(content)]
    words_counter = Counter(words)
    kw_dc[year] = np.array(words_counter.most_common(20))[:, 0]

In [15]:
pd.DataFrame(kw_dc).to_csv("kw_only.csv", encoding='euc-kr', index=False)

In [16]:
# only prop
prop_ls = counter_dc.copy()

for i in range(2000, 2022, 1):
    for j in range(len(prop_ls[i])):
        prop_ls[i][j] = round(prop_ls[i][j][1]/COUNT[i], 3)

In [17]:
pd.DataFrame(prop_ls).to_csv("prop_only.csv", encoding='euc-kr', index=False)